In [1]:
import torch
import torch.nn as nn

In [2]:
input_dim = 5
hidden_dim = 10
n_layers = 1

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

In [5]:
batch_size = 1
seq_len = 1

inp = torch.randn(batch_size, seq_len, input_dim)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)

In [9]:
print(inp)
print (hidden_state)
print (cell_state)
print (hidden)

tensor([[[-0.2558,  0.2597,  1.1702,  0.4267,  0.0070]]])
tensor([[[ 1.7694,  0.4158,  1.9819,  0.7717,  1.5680,  0.4820,  0.3440,
           1.5060,  0.1762, -0.1831]]])
tensor([[[-0.6529, -1.3589,  0.3999,  0.2631, -0.1287, -0.1853,  0.4635,
           0.5417,  0.9315,  0.6401]]])
(tensor([[[ 1.7694,  0.4158,  1.9819,  0.7717,  1.5680,  0.4820,  0.3440,
           1.5060,  0.1762, -0.1831]]]), tensor([[[-0.6529, -1.3589,  0.3999,  0.2631, -0.1287, -0.1853,  0.4635,
           0.5417,  0.9315,  0.6401]]]))


In [10]:
out, hidden = lstm_layer(inp, hidden)

In [11]:
print("Output shape: ", out.shape)
print("Hidden: ", hidden)

Output shape:  torch.Size([1, 1, 10])
Hidden:  (tensor([[[-0.3424, -0.2964,  0.1587,  0.0031, -0.1170, -0.1018,  0.0795,
           0.3849,  0.3271,  0.1692]]], grad_fn=<StackBackward>), tensor([[[-0.5722, -0.8397,  0.4341,  0.0051, -0.3145, -0.1987,  0.0891,
           0.5270,  1.0386,  0.4149]]], grad_fn=<StackBackward>))


In [12]:
seq_len = 3
inp = torch.randn(batch_size, seq_len, input_dim)
out, hidden = lstm_layer(inp, hidden)
print(out.shape)

torch.Size([1, 3, 10])


In [13]:
# Obtaining the last output
out = out.squeeze()[-1, :]
print(out.shape)

torch.Size([10])


In [25]:
import bz2
from collections import Counter
import re
import nltk
import numpy as np
#nltk.download('punkt')

train_file = bz2.BZ2File('./train.ft.txt.bz2')
test_file = bz2.BZ2File('./test.ft.txt.bz2')

train_file = train_file.readlines()
test_file = test_file.readlines()

In [26]:
num_train = 800000  # We're training on the first 800,000 reviews in the dataset
num_test = 200000  # Using 200,000 reviews from test set

train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]

In [27]:
# Extracting labels from sentences
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]

test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]

# Some simple cleaning of data
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

# Modify URLs to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [28]:
print (test_sentences[2])

batteries died within a year ...: i bought this charger in jul 0000 and it worked ok for a while. the design is nice and convenient. however, after about a year, the batteries would not hold a charge. might as well just get alkaline disposables, or look elsewhere for a charger that comes with batteries that have better staying power.


In [29]:
words = Counter()  # Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_sentences):
    # The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word.lower()])  # Converting all the words to lowercase
        train_sentences[i].append(word)
    if i%20000 == 0:
        print(str((i*100)/num_train) + "% done", end='\r')
print("100% done")

100% donee


In [30]:
# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

In [40]:
print (word2idx["professional"])
print (idx2word[4001])

1331
gym


In [41]:
for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]

In [42]:
print (train_sentences[1])

[3, 96, 948, 131, 8, 272, 2, 13, 5, 122, 154, 7, 188, 10, 258, 765, 17, 11, 12, 3, 96, 66169, 948, 155, 6, 5, 1528, 17, 5, 295, 485, 7, 274, 8, 3046, 7, 268, 2, 11, 14, 28, 0, 12, 121238, 70542, 21, 1931, 1074, 2, 3, 126, 12, 2152, 6, 5, 122, 105, 526, 8, 9, 16, 151, 129, 6, 136, 1287, 404, 5708, 8, 152736, 215, 3943, 25, 11, 12, 249, 14735, 5, 187, 158, 4, 22, 39, 19, 30, 191, 8, 107, 111, 99, 16, 11, 87, 144, 4, 11, 12, 3, 75, 31, 17, 5, 243, 51, 34, 177, 168, 2612, 2]


In [43]:
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200  # The length that the sentences will be padded/shortened to

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

# Converting our labels into numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [46]:
print (train_sentences[5])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0    52  1347  1074    13     5   112   283   253   111
    10    19   250   681     3    70     8    60    11    27   539     3
   211    48   322   299     4     6   301    48   322     7   174    10
     3   488   193     2     6   995    19    94  1754    10     9    53
    20     4 70542    21   126  8692  2193     8     3  1845    10   168
   540  1184    10     3   293   535   571   492 15054    10    55   966
     6   283     7   174   157    46     5    27    26    52  1854  1862
    44     4    35    10   110    30 66170     6 22517  2459     4    11
   948    12    31     5  7190    19    19    67    20   908     2     9
    58   309    16   168  3075   167    49  5526     6  5393    46  2080
     3     0    53 62446   390    44     4     8  3779     6    61  2533
    46  2442   477    44     4     8  5534  1356  6

In [47]:
split_frac = 0.5 # 50% validation, 50% test
split_id = int(split_frac * len(test_sentences))
val_sentences, test_sentences = test_sentences[:split_id], test_sentences[split_id:]
val_labels, test_labels = test_labels[:split_id], test_labels[split_id:]

In [48]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

In [49]:
train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

In [50]:
print (train_data[0])

(tensor([     0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,

In [51]:
batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [52]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
print (is_cuda)

False


In [53]:
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [54]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [55]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)

lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [56]:
epochs = 2
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

Epoch: 1/2... Step: 1000... Loss: 0.213796... Val Loss: 0.175634
Validation loss decreased (inf --> 0.175634).  Saving model ...
Epoch: 1/2... Step: 2000... Loss: 0.134035... Val Loss: 0.170129
Validation loss decreased (0.175634 --> 0.170129).  Saving model ...


KeyboardInterrupt: 

In [ ]:
print ("hello")